In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')
from utils.data_utils import read_training_data, read_model_zoo
from utils.pipeline_utils import run_cv
from models.tfidf_model import TfidfModel
from models.bert_model import BertModel,BertModelwithMaskedLM

In [3]:
df = read_training_data()
df.head()

,id,text,is_offensive,target,text_len,target_label,public_fold,private_fold,pred
0,81c11060-a240-4d54-841b-9e2916039e85,çürük dişli,1,INSULT,11,4,1,0,OTHER
1,be80ebbf-b322-4c3b-afa1-94932ea80731,bu adamın islama ve müslümanlara verdiği zarar...,1,RACIST,67,3,3,0,OTHER
2,f99e2513-83ed-4076-ac72-b9e2cff3f049,erkekler zora gelmez,1,SEXIST,20,2,2,0,OTHER
3,83ed2b2e-b815-4f36-9fc4-80a9050cf2d0,utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1,PROFANITY,65,1,4,0,OTHER
4,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER,46,0,4,0,OTHER


In [4]:
model_params={"model_path": 'dbmdz/bert-base-turkish-128k-uncased',
              "epochs": 3,
              "batch_size": 16,
              "tokenizer_max_len": 40,
              "learning_rate": 5e-5}

In [5]:
experiment_name = f'{model_params["model_path"]} | epochs: {model_params["epochs"]} | batch_size: {model_params["batch_size"]} | max_len: {model_params["tokenizer_max_len"]}' 

In [6]:
experiment_name

'dbmdz/bert-base-turkish-128k-uncased | epochs: 3 | batch_size: 16 | max_len: 40'

In [7]:
run_cv(model_obj=BertModelwithMaskedLM,
       model_params=model_params,
       input_df=df,
       fold_col='public_fold',
       x_col='text',
       y_col='target_label',
       experiment_name=experiment_name,
       add_to_zoo=False,
       is_nn=True)


******************************
Started CV Training
******************************
Experiment: 'dbmdz/bert-base-turkish-128k-uncased | epochs: 3 | batch_size: 16 | max_len: 40'
Fold: 'public_fold'
Update Zoo: 'False'
******************************



Training.. Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificatio

Batch:   0%|          | 0/621 [00:00<?, ?it/s]

Epoch 1, Validation Accuracy: 0.24971, Validation F1-Macro: 0.92491


...

Epoch 3, Validation Accuracy: 0.15874, Validation F1-Macro: 0.95542


Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificatio

Batch:   0%|          | 0/621 [00:00<?, ?it/s]

Epoch 1, Validation Accuracy: 0.23929, Validation F1-Macro: 0.92952


Batch:   0%|          | 0/621 [00:00<?, ?it/s]